In [3]:
# install and import package
!pip install  snownlp
!pip install  ggplot
import pandas
import requests
from snownlp import SnowNLP
from bs4 import BeautifulSoup


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [35]:
# 下載微博武漢肺炎相關貼文
def download_Cov_data(max):
    articles = []
    for i in range(1,max):
        res = requests.get('https://m.weibo.cn/api/container/getIndex?containerid=231522type%3D1%26q%3D%23%E6%AD%A6%E6%B1%89%E8%82%BA%E7%82%8E%23&page_type=searchall&page={}'.format(i))
        jd = res.json()
        articles.extend([rec['mblog'] for rec in jd['data']['cards'] if rec.get('mblog')])

    return pandas.DataFrame(articles)
# 撈1~50頁資料
df = download_Cov_data(50)


In [40]:
# 取出貼文資料
def parseArticle(e):
    soup = BeautifulSoup(e, 'html.parser')
    return soup.text

def parseQueryTime(e):
    return e.split('&qtime=')[1].replace("&","")

text_data=df['text'].map(parseArticle)
query_time_data=df['itemid'].map(parseQueryTime)

In [41]:
# 進行每編貼文的情緒分析

def getSentenment(e):
    s = SnowNLP(e)
    return s.sentiments

sentiments_data=text_data.map(getSentenment)

dict = {
    "查詢時間":query_time_data,
    "貼文內容": text_data,  
     "情緒分數": sentiments_data
       }

select_df = pandas.DataFrame(dict)

In [42]:
# 輸出分析結果至terminal
sorted_data= select_df.sort_values(by = '情緒分數',ascending = False)#
sorted_data

# 輸出分析結果至export_dataframe.csv檔
# 情緒分數>0  =>  正面情緒
# 情緒分數<=0  => 負面情緒
sorted_data.to_csv (r'export_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
